In [1]:
from web3 import Web3
import json
import pandas as pd
import time
import datetime

ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

if not web3.is_connected():
    print("❌ Could not connect. Make sure Ganache is running.")
    exit(1)
print("✅ Connected to Ganache.")

contract_address = "0xA93B691113295c0E1602F1fE341cC062286d3eeA"

abi_json = """
[
  {
    "inputs": [],
    "stateMutability": "nonpayable",
    "type": "constructor"
  },
  {
    "anonymous": false,
    "inputs": [
      {
        "indexed": false,
        "internalType": "uint256",
        "name": "timestamp",
        "type": "uint256"
      },
      {
        "indexed": false,
        "internalType": "string",
        "name": "rfid",
        "type": "string"
      },
      {
        "indexed": false,
        "internalType": "string",
        "name": "gpsSensor",
        "type": "string"
      },
      {
        "indexed": false,
        "internalType": "string",
        "name": "temperatureSensor",
        "type": "string"
      }
    ],
    "name": "DataStored",
    "type": "event"
  },
  {
    "inputs": [
      {
        "internalType": "string",
        "name": "_rfid",
        "type": "string"
      },
      {
        "internalType": "string",
        "name": "_gpsSensor",
        "type": "string"
      },
      {
        "internalType": "string",
        "name": "_temperatureSensor",
        "type": "string"
      }
    ],
    "name": "storeData",
    "outputs": [],
    "stateMutability": "nonpayable",
    "type": "function"
  },
  {
    "inputs": [
      {
        "internalType": "uint256",
        "name": "",
        "type": "uint256"
      }
    ],
    "name": "dataRecords",
    "outputs": [
      {
        "internalType": "uint256",
        "name": "timestamp",
        "type": "uint256"
      },
      {
        "internalType": "string",
        "name": "rfid",
        "type": "string"
      },
      {
        "internalType": "string",
        "name": "gpsSensor",
        "type": "string"
      },
      {
        "internalType": "string",
        "name": "temperatureSensor",
        "type": "string"
      }
    ],
    "stateMutability": "view",
    "type": "function"
  },
  {
    "inputs": [
      {
        "internalType": "uint256",
        "name": "index",
        "type": "uint256"
      }
    ],
    "name": "getRecord",
    "outputs": [
      {
        "internalType": "uint256",
        "name": "",
        "type": "uint256"
      },
      {
        "internalType": "string",
        "name": "",
        "type": "string"
      },
      {
        "internalType": "string",
        "name": "",
        "type": "string"
      },
      {
        "internalType": "string",
        "name": "",
        "type": "string"
      }
    ],
    "stateMutability": "view",
    "type": "function"
  },
  {
    "inputs": [],
    "name": "getTotalRecords",
    "outputs": [
      {
        "internalType": "uint256",
        "name": "",
        "type": "uint256"
      }
    ],
    "stateMutability": "view",
    "type": "function"
  },
  {
    "inputs": [],
    "name": "MAX_ENTRIES",
    "outputs": [
      {
        "internalType": "uint256",
        "name": "",
        "type": "uint256"
      }
    ],
    "stateMutability": "view",
    "type": "function"
  },
  {
    "inputs": [],
    "name": "owner",
    "outputs": [
      {
        "internalType": "address",
        "name": "",
        "type": "address"
      }
    ],
    "stateMutability": "view",
    "type": "function"
  }
]
"""

abi = json.loads(abi_json)

contract = web3.eth.contract(address=contract_address, abi=abi)

web3.eth.default_account = web3.eth.accounts[0]
print(f"✅ Connected to Smart Contract at {contract_address}")
print(f"Using default account: {web3.eth.default_account}\n")

df = pd.read_csv("logistics_tracking_data.csv")

print("CSV Preview:")
print(df.head(), "\n")

def send_iot_data(rfid: str, gps_sensor: str, temperature_sensor: str):
    """
    Sends a single IoT data record to the deployed smart contract.
    """
    
    txn = contract.functions.storeData(rfid, gps_sensor, temperature_sensor).transact({
        'from': web3.eth.default_account,
        'gas': 3_000_000
    })
    
    receipt = web3.eth.wait_for_transaction_receipt(txn)
    print(f"✅ Stored | RFID: {rfid} | GPS Sensor: {gps_sensor} | Temp Sensor: {temperature_sensor} | Txn: {receipt.transactionHash.hex()}")


print("⏳ Sending CSV rows to blockchain...\n")
for idx, row in df.iterrows():

    dev_id    = str(row["rfid"])
    dtype     = str(row["gps_sensor"])
    dvalue    = str(row["temperature_sensor"])

    send_iot_data(dev_id, dtype, dvalue)

    time.sleep(1)

print("\n✅ Finished sending all CSV rows.\n")

total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")

first = contract.functions.getRecord(0).call()
ts0, d0, t0, v0 = first
print("\nFirst stored record:")
print(f"  timestamp: {datetime.datetime.utcfromtimestamp(ts0).strftime('%Y-%m-%d %H:%M:%S UTC')}")
print(f"  rfid : {d0}")
print(f"  gpsSensor : {t0}")
print(f"  temperatureSensor: {v0}")

last_index = total_records - 1
last = contract.functions.getRecord(last_index).call()
tsL, dL, tL, vL = last
print("\nLast stored record:")
print(f"  timestamp: {datetime.datetime.utcfromtimestamp(tsL).strftime('%Y-%m-%d %H:%M:%S UTC')}")
print(f"  rfid : {dL}")
print(f"  gpsSensor : {tL}")
print(f"  temperatureSensor: {vL}")


✅ Connected to Ganache.
✅ Connected to Smart Contract at 0xA93B691113295c0E1602F1fE341cC062286d3eeA
Using default account: 0x4A09680C60e5D6f48d109c2cD984c56282C1DEc6

CSV Preview:
                    timestamp    rfid  gps_sensor  temperature_sensor
0  2025-05-08 06:37:34.786672  TRK716          61                20.4
1  2025-05-07 23:56:34.795947  TRK578          76                24.6
2  2025-05-08 07:54:34.796037  TRK434          63                23.1
3  2025-05-08 02:26:34.796065  TRK230          79                23.7
4  2025-05-07 22:51:34.796086  TRK593          62                23.8 

⏳ Sending CSV rows to blockchain...

✅ Stored | RFID: TRK716 | GPS Sensor: 61 | Temp Sensor: 20.4 | Txn: 116e66d8f35c90f1e3226b753b4246d0fb08b5d21b3e57f7a7fccd89cac8c1ce
✅ Stored | RFID: TRK578 | GPS Sensor: 76 | Temp Sensor: 24.6 | Txn: 60e6b237a517925cba8dad6dc937381a96d41efe0356bb9351eea58598fe7f93
✅ Stored | RFID: TRK434 | GPS Sensor: 63 | Temp Sensor: 23.1 | Txn: 550c0f67984fd148617b722d203

C:\Users\pc\AppData\Local\Temp\ipykernel_23328\2782536391.py:234: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print(f"  timestamp: {datetime.datetime.utcfromtimestamp(ts0).strftime('%Y-%m-%d %H:%M:%S UTC')}")
C:\Users\pc\AppData\Local\Temp\ipykernel_23328\2782536391.py:243: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print(f"  timestamp: {datetime.datetime.utcfromtimestamp(tsL).strftime('%Y-%m-%d %H:%M:%S UTC')}")


In [2]:
total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")

Total IoT records stored: 100


In [3]:
# Retrieve all IoT records
data = []
for i in range(total_records):
    record = contract.functions.getRecord(i).call()
    data.append({
        "timestamp": record[0],
        "rfid": record[1],
        "gps_sensor": record[2],
        "temperature_sensor": record[3]
    })

# Convert to a DataFrame
df = pd.DataFrame(data)

# Convert timestamp to readable format
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

# Display first few records
print(df.head())


            timestamp    rfid gps_sensor temperature_sensor
0 2025-06-10 00:59:18  TRK716         61               20.4
1 2025-06-10 00:59:19  TRK578         76               24.6
2 2025-06-10 00:59:21  TRK434         63               23.1
3 2025-06-10 00:59:22  TRK230         79               23.7
4 2025-06-10 00:59:23  TRK593         62               23.8


In [4]:
import numpy as np


In [8]:
# Retrieve all IoT records
data = []
for i in range(total_records):
    record = contract.functions.getRecord(i).call()
    data.append({
        "timestamp": record[0],
        "rfid": record[1],
        "gps_sensor": record[2],
        "temperature_sensor": record[3]
    })

# Convert to DataFrame
df = pd.DataFrame(data)

# Convert timestamp to readable format
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

# Extract temperature (float)
df["numeric_temperature"] = df["temperature_sensor"].str.extract(r'(\d+\.?\d*)').astype(float)

# Extract latitude and longitude separately from gps_sensor
gps_split = df["gps_sensor"].str.extract(r'(-?\d+\.?\d*),\s*(-?\d+\.?\d*)')

# Fill any missing values with 0
df.fillna(0, inplace=True)

# Display the full cleaned dataset
print(df.head())


            timestamp    rfid gps_sensor temperature_sensor  \
0 2025-06-10 00:59:18  TRK716         61               20.4   
1 2025-06-10 00:59:19  TRK578         76               24.6   
2 2025-06-10 00:59:21  TRK434         63               23.1   
3 2025-06-10 00:59:22  TRK230         79               23.7   
4 2025-06-10 00:59:23  TRK593         62               23.8   

   numeric_temperature  
0                 20.4  
1                 24.6  
2                 23.1  
3                 23.7  
4                 23.8  


In [ ]:
# DO THIS IF NEEDED! PROVIDED CSV IS ALREADY CLEANED.
# # Save cleaned IoT data to a CSV file
# df.to_csv("cleaned_iot_data.csv", index=False)


# print("✅ Cleaned IoT data saved successfully as cleaned_iot_data.csv")

✅ Cleaned IoT data saved successfully as cleaned_iot_data.csv
